In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
set_base_result_dir = '../main_Run_63/'
set_new_result_dir = '../main_Run_66/'

indiv_tour_df_base = pd.read_csv(os.path.join(set_base_result_dir, 'indivTourData_1.csv'))

indiv_tour_df_new = pd.read_csv(os.path.join(set_new_result_dir, 'indivTourData_1.csv'))
distance_skim = pd.read_csv(os.path.join(set_base_result_dir, 'da_am_skims.csv'))
distance_skim = distance_skim[(distance_skim['dist'].notnull())&(distance_skim['dist']<10000)]

In [4]:

def tour_purpose_category(x):
    if x in ['work_med', 'work_very high', 'work_high','work_low']:
        return 'Work'
    elif x in ['school_grade','school_high']:
        return 'School'
    elif x in ['atwork_eat','atwork_maint','atwork_business']:
        return 'At-Work'
    elif x == 'shopping':
        return 'Shopping'
    elif x in ['escort_no kids','escort_kids']:
        return 'Escorting'
    elif x=='university':
        return 'University'
    elif x=='eatout':
        return 'Eat Out'
    elif x=='social':
        return 'Social'
    elif x=='othdiscr':
        return 'Discretionary'
    elif x=='othmaint':
        return 'Maintenance'
    else:
        return None

indiv_tour_df_base['Tour Purpose']=indiv_tour_df_base['tour_purpose'].apply(tour_purpose_category) 
indiv_tour_df_base['person_id']=indiv_tour_df_base['person_id'].astype(str)
indiv_tour_df_base['tour_id']=indiv_tour_df_base['tour_id'].astype(str)
indiv_tour_df_base['unique_tour_id']=indiv_tour_df_base['person_id']+'-'+indiv_tour_df_base['tour_id']

person_df_ws_loc_base=indiv_tour_df_base.merge(distance_skim[['origin','destination','dist']],
                                 how='left',
                                 left_on=['orig_taz','dest_taz'],
                                 right_on=['origin','destination'])

In [6]:
indiv_tour_df_new['Tour Purpose']=indiv_tour_df_new['tour_purpose'].apply(tour_purpose_category) 
indiv_tour_df_new['person_id']=indiv_tour_df_new['person_id'].astype(str)
indiv_tour_df_new['tour_id']=indiv_tour_df_new['tour_id'].astype(str)
indiv_tour_df_new['unique_tour_id']=indiv_tour_df_new['person_id']+'-'+indiv_tour_df_new['tour_id']

person_df_ws_loc_new=indiv_tour_df_new.merge(distance_skim[['origin','destination','dist']],
                                 how='left',
                                 left_on=['orig_taz','dest_taz'],
                                 right_on=['origin','destination'])

In [7]:
distance_bin=range(0,150)

tour_length_avg_base =  person_df_ws_loc_base.groupby('Tour Purpose').agg(average_distance_model = ('dist','mean')).reset_index()
tour_length_avg_new =  person_df_ws_loc_new.groupby('Tour Purpose').agg(average_distance_model = ('dist','mean')).reset_index()

In [10]:
import math

In [11]:
person_df_ws_loc_base['upper_limit'] = person_df_ws_loc_base['dist'].apply(lambda x: math.ceil(x))
person_df_ws_loc_new['upper_limit'] = person_df_ws_loc_new['dist'].apply(lambda x: math.ceil(x))

person_share_base= person_df_ws_loc_base.groupby(['Tour Purpose','upper_limit']).agg(Unique_Tours=('unique_tour_id','nunique')).reset_index()
person_share_base['Total_Tours']=person_share_base.groupby('Tour Purpose')['Unique_Tours'].transform(sum)
person_share_base['main_Run_63']=person_share_base['Unique_Tours']/person_share_base['Total_Tours']
person_share_base=person_share_base.rename(columns={'Tour Purpose':'dimension_01_value'})
person_share_base

,dimension_01_value,upper_limit,Unique_Tours,Total_Tours,main_Run_63
0,At-Work,1,28334,84638,0.334767
1,At-Work,2,16264,84638,0.192160
2,At-Work,3,9803,84638,0.115823
3,At-Work,4,6689,84638,0.079031
4,At-Work,5,4347,84638,0.051360
...,...,...,...,...,...
808,Work,127,4,314223,0.000013
809,Work,128,1,314223,0.000003
810,Work,129,1,314223,0.000003
811,Work,131,1,314223,0.000003


In [14]:
person_share_new= person_df_ws_loc_new.groupby(['Tour Purpose','upper_limit']).agg(Unique_Tours=('unique_tour_id','nunique')).reset_index()
person_share_new['Total_Tours']=person_share_new.groupby('Tour Purpose')['Unique_Tours'].transform(sum)
person_share_new['main_Run_66']=person_share_new['Unique_Tours']/person_share_new['Total_Tours']
person_share_new=person_share_new.rename(columns={'Tour Purpose':'dimension_01_value'})
person_share_new

,dimension_01_value,upper_limit,Unique_Tours,Total_Tours,main_Run_66
0,At-Work,1,29108,85001,0.342443
1,At-Work,2,16438,85001,0.193386
2,At-Work,3,10008,85001,0.117740
3,At-Work,4,6472,85001,0.076140
4,At-Work,5,4153,85001,0.048858
...,...,...,...,...,...
743,Work,102,2,314818,0.000006
744,Work,103,3,314818,0.000010
745,Work,105,3,314818,0.000010
746,Work,108,1,314818,0.000003


In [15]:
comparison = pd.merge(person_share_base, person_share_new, on=['dimension_01_value','upper_limit'], how='outer')

comparison

,dimension_01_value,upper_limit,Unique_Tours_x,Total_Tours_x,main_Run_63,Unique_Tours_y,Total_Tours_y,main_Run_66
0,At-Work,1,28334.0,84638.0,0.334767,29108.0,85001.0,0.342443
1,At-Work,2,16264.0,84638.0,0.192160,16438.0,85001.0,0.193386
2,At-Work,3,9803.0,84638.0,0.115823,10008.0,85001.0,0.117740
3,At-Work,4,6689.0,84638.0,0.079031,6472.0,85001.0,0.076140
4,At-Work,5,4347.0,84638.0,0.051360,4153.0,85001.0,0.048858
...,...,...,...,...,...,...,...,...
851,Social,86,NaN,NaN,NaN,1.0,32620.0,0.000031
852,Social,90,NaN,NaN,NaN,3.0,32620.0,0.000092
853,University,60,NaN,NaN,NaN,3.0,26910.0,0.000111
854,University,67,NaN,NaN,NaN,3.0,26910.0,0.000111
